Лабораторная работа №3

In [ ]:
import numpy as np

cup1 =   [0,1,0,0,0,
           1,0,1,1,0,
           1,1,1,0,1,
           1,0,1,1,0,
           1,1,1,0,0]

cup2 =   [0,1,0,0,0,
           1,0,1,1,0,
           1,1,1,0,1,
           1,0,1,1,0,
           1,1,1,0,0]

cup3 =   [0,1,0,0,0,
           1,0,1,1,0,
           1,1,1,0,1,
           1,0,1,1,0,
           1,1,1,0,0]

cup4 =   [0,1,0,0,0,
           1,0,1,1,0,
           1,1,1,0,1,
           1,0,1,1,0,
           1,1,1,0,0]

cup5 =   [0,1,0,0,0,
           1,0,1,1,0,
           1,1,1,0,1,
           1,0,1,1,0,
           1,1,1,0,0]

butle1 =  [0,1,1,0,0,
            0,1,1,0,0,
            1,0,0,1,0,
            1,0,0,1,0,
            1,1,1,1,0]

butle2 =  [0,1,1,0,0,
            0,1,1,0,0,
            1,0,0,1,0,
            1,0,0,1,0,
            1,1,1,1,0]

butle3 =  [0,1,1,0,0,
            0,1,1,0,0,
            1,0,0,1,0,
            1,0,0,1,0,
            1,1,1,1,0]

butle4 =  [0,1,1,0,0,
            0,1,1,0,0,
            1,0,0,1,0,
            1,0,0,1,0,
            1,1,1,1,0]

butle5 =  [0,1,1,0,0,
            0,1,1,0,0,
            1,0,0,1,0,
            1,0,0,1,0,
            1,1,1,1,0]

def get_cup():
    return np.array([cup1, cup2, cup3, cup4, cup5])

def get_butle():
    return np.array([butle1, butle2, butle3, butle4, butle5])

def get_conv_numbers(number):
    if number == 1:
        return [[np.sum(row) for row in num] for num in get_butle().reshape(5, 5, 5)]
    elif number == 0:
        return [[np.sum(row) for row in num] for num in get_cup().reshape(5, 5, 5)]
    else:
        raise Exception("Нет такой цифры, введите либо чашку либо бутылку")

def get_conv_number(number):
    return np.array([np.sum(row) for row in number.reshape(5, 5)])

In [ ]:
import numpy as np
from sklearn.metrics import precision_score, accuracy_score

def add_bias_feature(a):
    a_extended = np.zeros((a.shape[0],a.shape[1]+1))
    a_extended[:,:-1] = a
    a_extended[:,-1] = int(1)  
    return a_extended

class SVM(object):
    def __init__(self, etha=0.01, alpha=0.1, epochs=200):
        self._epochs = epochs
        self._etha = etha
        self._alpha = alpha
        self._w = None # Веса модели
        self.history_w = [] # История изменения весов для показа
        self.train_errors = None # Ошибки обучения модели
        self.val_errors = None # Ошибки проверки качества модели
        self.train_loss = None # Значение функции потерь модели при обучении
        self.val_loss = None # Значение функции потерь модели при проверке

    def fit(self, X_train, Y_train, X_val, Y_val, verbose=False): #arrays: X; Y =-1,1
        '''
        Метод обучения модели, значения массивов y принимают значения либо 1 либо -1
        1 - если на выходе должна быть еденица, -1 если ноль 
        '''
        if len(set(Y_train)) != 2 or len(set(Y_val)) != 2: # Проверка, что бы количество классов было равно 2 (бин классификация)
            raise ValueError("Number of classes in Y is not equal 2!")

        X_train = add_bias_feature(X_train) # Добавление признака смещения в каждый столбец
        X_val = add_bias_feature(X_val)
        self._w = np.random.normal(loc=0, scale=0.05, size=X_train.shape[1]) # Инициализируем веса случайными значениями
        self.history_w.append(self._w) # Записываем начальные веса в историю весов
        # Инициализируем списки
        train_errors = []
        val_errors = []
        train_loss_epoch = []
        val_loss_epoch = []
        # Начало тренировки модели
        for epoch in range(self._epochs):
            tr_err = 0
            val_err = 0
            tr_loss = 0
            val_loss = 0
            for i,x in enumerate(X_train): # Индексируем элементы обучающей выборки
                margin = Y_train[i]*np.dot(self._w,X_train[i]) # Находим значение отступа модели
                if margin >= 1: # классифицируем верно, если отступ элемента больше радиуса полосы модели
                    self._w -= self._etha*self._alpha*self._w/self._epochs # Изменяем веса пропорционально эпохе обучения
                    tr_loss += self.soft_margin_loss(X_train[i],Y_train[i]) # Увеличения значерия потерь
                else: # классифицируем неверно или попадаем на полосу разделения при 0<m<1
                    self._w += self._etha*(Y_train[i]*X_train[i] - self._alpha*self._w/self._epochs)
                    tr_err += 1
                    tr_loss += self.soft_margin_loss(X_train[i],Y_train[i])
                self.history_w.append(self._w)
            for i,x in enumerate(X_val): # Цикл проверки качества модели
                val_loss += self.soft_margin_loss(X_val[i], Y_val[i])
                val_err += (Y_val[i]*np.dot(self._w,X_val[i])<1).astype(int)
            if verbose and epoch % 20 == 0:
                print('epoch {}. Errors={}. Mean Hinge_loss={}'\
                      .format(epoch,val_err, val_loss))
            train_errors.append(tr_err)
            val_errors.append(val_err)
            train_loss_epoch.append(tr_loss)
            val_loss_epoch.append(val_loss)
        self.history_w = np.array(self.history_w)    
        self.train_errors = np.array(train_errors)
        self.val_errors = np.array(val_errors)
        self.train_loss = np.array(train_loss_epoch)
        self.val_loss = np.array(val_loss_epoch)                    

    def predict(self, X:np.array) -> np.array:
        y_pred = []
        X_extended = add_bias_feature(X)
        for i in range(len(X_extended)):
            y_pred.append(np.sign(np.dot(self._w,X_extended[i])))
        return np.array(y_pred)         

    def hinge_loss(self, x, y):
        return max(0,1 - y*np.dot(x, self._w))

    def soft_margin_loss(self, x, y):
        return self.hinge_loss(x,y)+self._alpha*np.dot(self._w, self._w)


svm = SVM(epochs=1000)
svm.fit(np.vstack([get_conv_numbers(0), get_conv_numbers(1)]), np.array([-1,-1,-1,-1,-1,1,1,1,1,1]),
        np.vstack([get_conv_numbers(0)[3:], get_conv_numbers(1)[3:]]), np.array([-1,-1,-1,1,1,1]),
        verbose=True)
# Предсказать число, если чашка то output = -1, иначе 1
number =  np.array([0,1,0,0,0,
                    1,0,1,1,0,
                    1,1,1,0,1,
                    1,0,1,1,0,
                    1,1,1,0,0])

y_pred = svm.predict(np.array([get_conv_number(number)]))

print("Class: " + str(svm.predict(np.array([get_conv_number(number)]))))
print("Accuracy: " + str(accuracy_score(cup3, number)))

epoch 0. Errors=4. Mean Hinge_loss=5.29849363743798
epoch 20. Errors=1. Mean Hinge_loss=2.2874965237610754
epoch 40. Errors=1. Mean Hinge_loss=2.287188537047287
epoch 60. Errors=1. Mean Hinge_loss=2.2868806321258965
epoch 80. Errors=1. Mean Hinge_loss=2.286572808972467
epoch 100. Errors=1. Mean Hinge_loss=2.2862650675625673
epoch 120. Errors=1. Mean Hinge_loss=2.2859574078717806
epoch 140. Errors=1. Mean Hinge_loss=2.2856498298756946
epoch 160. Errors=1. Mean Hinge_loss=2.2853423335499063
epoch 180. Errors=1. Mean Hinge_loss=2.28503491887002
epoch 200. Errors=1. Mean Hinge_loss=2.2847275858116487
epoch 220. Errors=1. Mean Hinge_loss=2.284420334350415
epoch 240. Errors=1. Mean Hinge_loss=2.2841131644619477
epoch 260. Errors=1. Mean Hinge_loss=2.2838060761218837
epoch 280. Errors=1. Mean Hinge_loss=2.283499069305868
epoch 300. Errors=1. Mean Hinge_loss=2.283192143989557
epoch 320. Errors=1. Mean Hinge_loss=2.2828853001486045
epoch 340. Errors=1. Mean Hinge_loss=2.2825785377586865
epoch 3